<a href="https://colab.research.google.com/github/sujitkoley9/Sentiment-Analysis/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <center>Sentiment Analysis </center> </h1>
<b>Useful link: </b>


1. https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
2. https://realpython.com/python-keras-text-classification/

3. https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f

4. https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/


# 1. **Importing packages**



In [0]:
from datetime import date, timedelta,datetime
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

import gensim
from gensim.test.utils import get_tmpfile
from gensim.summarization import keywords

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV


from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# 2 . **Reading required files**

In [0]:
movie_df = pd.read_csv('amazon_cells_labelled.txt', names=['review', 'sentiment'],sep='\t')

<1000x1847 sparse matrix of type '<class 'numpy.float64'>'
	with 9130 stored elements in Compressed Sparse Row format>

# 1. **Logistic Regression**


In [23]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']

vectorizer = TfidfVectorizer(max_df =1.0,min_df=1,analyzer='word' )
X = vectorizer.fit_transform(X)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# 2. **Neural network without word embedding**


In [33]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']

vectorizer = TfidfVectorizer(max_df =1.0,min_df=1,analyzer='word' )
X = vectorizer.fit_transform(X)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

# Neural Network 
classifier  = Sequential()
classifier.add(Dense(10,input_shape=(X_train.shape[1],), activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))

# model summary
classifier.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                18480     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 11        
Total params: 18,491
Trainable params: 18,491
Non-trainable params: 0
_________________________________________________________________


In [45]:
# compile the model
classifier.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


# Train the model
history = classifier.fit(X_train, y_train,
                    epochs=100,
                    verbose=1 ,
                    validation_data=(X_test, y_test),
                    batch_size=X_train.shape[0])

Train on 750 samples, validate on 250 samples
Epoch 1/100
750/750 [==============================] - 0s 123us/sample - loss: 7.8678e-09 - accuracy: 1.0000 - val_loss: 1.1842 - val_accuracy: 0.7920
Epoch 2/100
750/750 [==============================] - 0s 15us/sample - loss: 7.6294e-09 - accuracy: 1.0000 - val_loss: 1.1819 - val_accuracy: 0.7920
Epoch 3/100
750/750 [==============================] - 0s 15us/sample - loss: 7.9473e-09 - accuracy: 1.0000 - val_loss: 1.1836 - val_accuracy: 0.7920
Epoch 4/100
750/750 [==============================] - 0s 15us/sample - loss: 8.1857e-09 - accuracy: 1.0000 - val_loss: 1.1831 - val_accuracy: 0.7920
Epoch 5/100
750/750 [==============================] - 0s 12us/sample - loss: 7.6294e-09 - accuracy: 1.0000 - val_loss: 1.1823 - val_accuracy: 0.7920
Epoch 6/100
750/750 [==============================] - 0s 12us/sample - loss: 7.0731e-09 - accuracy: 1.0000 - val_loss: 1.1831 - val_accuracy: 0.7920
Epoch 7/100
750/750 [==============================] 

In [46]:
loss, accuracy = classifier.evaluate(X_test, y_test, verbose=False)
print("Accuracy:", accuracy)

Accuracy: 0.792
